In [33]:
#Pandas: Possui inúmeras funções e comandos para importar arquivos, analisar dados, tratar dados, etc.
import pandas as pd # importando a biblioteca pandas

#Time: Possui uma série de funções e comandos para trabalharmos com tempo
import time # importando a biblioteca time

#os: Possui uma série de funções e comandos para trabalharmos com arquivos
import os # importando a biblioteca os

#XlsxWriter: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import xlsxwriter # importando a biblioteca xlsxwriter

#Locale: Possui uma série de funções e comandos para trabalharmos com formatação de moeda
import locale # importando a biblioteca locale

#Math: Possui uma série de funções e comandos para trabalharmos com matemática
import math # importando a biblioteca math

#Warnings: Possui detalhes sobre os avisos e alertas que aparecem, porém podemos utiliza-lo também para que os alertas de
#futuras atualizações e metodos depreciados não sejam exibidos
import warnings
warnings.filterwarnings("ignore") 
tempo_inicial = time.time() # tempo inicial para calcular o tempo de execução do código

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8') # definindo o locale para pt_BR
from datetime import datetime # Utilizado para trabalhar com datas
pd.set_option('display.max_columns', None) # Utilizado para exibir todas as colunas
pd.set_option('display.max_rows', None) # Utilizado para exibir todas as linhas

## Importação das Planilhas

### ABA 1

In [34]:
pasta = 'APROVADO'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Ident. Fila na UF'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo_com_extensao = os.path.basename(caminho_arquivo)  # Obtém o nome do arquivo com a extensão
        nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo_com_extensao)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo_sem_extensao  # Adiciona uma coluna 'Arquivo' com o nome do arquivo sem a extensão
        df_aba1['STATUS'] = os.path.dirname(caminho_arquivo)  # Adiciona uma coluna 'Pasta' com o nome da pasta onde está o arquivo
        dfs_dict[nome_arquivo_sem_extensao] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba1 = pd.concat(dfs_dict.values(), ignore_index=True)

In [35]:
df_planilha_aba1.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA':'CÓDIGO DO PROCEDIMENTO NO SIGTAP','Unnamed: 1':'PROCEDIMENTO CIRÚRGICO',
                                 'Unnamed: 2':'QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22', 'Unnamed: 3':'Redução do Tamanho da Fila (%)', 'Unnamed: 4':'Prazo para reduzir o % proposto (em meses)', 
                                 'Unnamed: 5':'Qtde de cirurgias a serem feitas no prazo pactuado', 'Unnamed: 6':'SQ (CODIGO Interno', 'UF':'UF'} , inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'

df_planilha_aba1.drop([0,1], inplace=True) # Exclui as linhas 0 e 1
df_planilha_aba1.drop('Unnamed: 7', axis=1, inplace=True) # Removendo coluna 'Unnamed: 7'
df_planilha_aba1.dropna(subset=['PROCEDIMENTO CIRÚRGICO'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1.drop(df_planilha_aba1[df_planilha_aba1['PROCEDIMENTO CIRÚRGICO'] == 'TOTAL'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1.drop(df_planilha_aba1[df_planilha_aba1['PROCEDIMENTO CIRÚRGICO'] == 'PROCEDIMENTO CIRÚRGICO'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1['QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22'] = df_planilha_aba1['QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22'].astype(int) # Converte a coluna 'Qtde de cirurgias a serem feitas no prazo pactuado' para inteiro
df_planilha_aba1['Redução do Tamanho da Fila (%)'] = df_planilha_aba1['Redução do Tamanho da Fila (%)'].astype(float) # Converte a coluna 'Redução do Tamanho da Fila (%)' para float
#df_planilha_aba1['Qtde de cirurgias a serem feitas no prazo pactuado'] = round(df_planilha_aba1['QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22'] * df_planilha_aba1['Redução do Tamanho da Fila (%)']).astype(int) # Cria uma nova coluna 'Qtde de cirurgias a serem feitas no prazo pactuado2' com o resultado da multiplicação da coluna 'QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22' pela coluna 'Redução do Tamanho da Fila (%)'
df_planilha_aba1['Qtde de cirurgias a serem feitas no prazo pactuado'] = df_planilha_aba1['QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22'] * df_planilha_aba1['Redução do Tamanho da Fila (%)']
df_planilha_aba1['Qtde de cirurgias a serem feitas no prazo pactuado'] = df_planilha_aba1['Qtde de cirurgias a serem feitas no prazo pactuado'].apply(math.ceil).astype(int) # Converte a coluna 'Qtde de cirurgias a serem feitas no prazo pactuado' para inteiro


In [36]:
df_planilha_aba1.head(10)

,CÓDIGO DO PROCEDIMENTO NO SIGTAP,PROCEDIMENTO CIRÚRGICO,QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22,Redução do Tamanho da Fila (%),Prazo para reduzir o % proposto (em meses),Qtde de cirurgias a serem feitas no prazo pactuado,SQ (CODIGO Interno,UF,STATUS
2,401020029,ENXERTO DERMO-EPIDERMICO,9,0.0,NaN,0,100001,ACRE,APROVADO
3,401020037,ENXERTO LIVRE DE PELE TOTAL,14,0.0,NaN,0,100002,ACRE,APROVADO
4,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",20,0.0,NaN,0,100003,ACRE,APROVADO
5,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,17,0.0,NaN,0,100004,ACRE,APROVADO
6,401020061,EXERESE DE CISTO BRANQUIAL,28,0.0,NaN,0,100005,ACRE,APROVADO
7,401020070,EXERESE DE CISTO DERMOIDE,138,0.0,NaN,0,100006,ACRE,APROVADO
8,401020088,EXERESE DE CISTO SACRO-COCCIGEO,1,0.0,NaN,0,100007,ACRE,APROVADO
9,401020096,EXERESE DE CISTO TIREOGLOSSO,18,1.0,9,18,100008,ACRE,APROVADO
10,401020100,EXTIRPACAO E SUPRESSAO DE LESAO DE PELE E DE T...,10,0.0,NaN,0,100009,ACRE,APROVADO
11,401020150,TRATAMENTO CIRURGICO DO SINUS PRE-AURICULAR,8,0.0,NaN,0,100010,ACRE,APROVADO


### ABA 2

In [37]:
pasta = 'APROVADO'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Ident. CNES e Proced.'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo_com_extensao = os.path.basename(caminho_arquivo)  # Obtém o nome do arquivo com a extensão
        nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo_com_extensao)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo_sem_extensao  # Adiciona uma coluna 'Arquivo' com o nome do arquivo sem a extensão
        df_aba1['STATUS'] = os.path.dirname(caminho_arquivo)  # Adiciona uma coluna 'Pasta' com o nome da pasta onde está o arquivo
        dfs_dict[nome_arquivo_sem_extensao] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba2 = pd.concat(dfs_dict.values(), ignore_index=True)

In [38]:
# Tratamento dos dados da aba 'Ident. CNES e Proced.'
df_planilha_aba2.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES':'CNES','Unnamed: 1':'ESTABELECIMENTO','Unnamed: 2':'CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP', 
                                 'Unnamed: 3':'PROCEDIMENTO CIRÚRGICO (PRINCIPAL)', 'Unnamed: 4':'COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENTO PRINCIPAL (Percentual)', 'Unnamed: 5':'GESTÃO DO SERVIÇO',
                                 'Unnamed: 6':'CODIGO DA NATUREZA JURÍDICA','Unnamed: 7':'NATUREZA JURÍDICA', 'Unnamed: 8':'POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?',
                                 'Unnamed: 9':'Identificação do Nome do Estabelecimento que não apareceu nesta planilha (coluna B)','Unnamed: 10':'SQ (CODIGO Interno'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'

df_planilha_aba2.drop([0,1], inplace=True) # Exclui as linhas 0 e 1
df_planilha_aba2.dropna(subset=['ESTABELECIMENTO'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba2.dropna(subset=['CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba2.drop(df_planilha_aba2[df_planilha_aba2['ESTABELECIMENTO'] == 'ESTABELECIMENTO'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'


In [39]:
df_planilha_aba2.head(10)


,CNES,ESTABELECIMENTO,CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP,PROCEDIMENTO CIRÚRGICO (PRINCIPAL),COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENTO PRINCIPAL (Percentual),GESTÃO DO SERVIÇO,CODIGO DA NATUREZA JURÍDICA,NATUREZA JURÍDICA,POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?,Identificação do Nome do Estabelecimento que não apareceu nesta planilha (coluna B),SQ (CODIGO Interno,UF,STATUS
2,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060135,HISTERECTOMIA TOTAL,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200001,ACRE,APROVADO
3,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060186,LAQUEADURA TUBARIA,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200002,ACRE,APROVADO
4,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060046,CURETAGEM SEMIOTICA C/ OU S/ DILATACAO DO COLO...,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200003,ACRE,APROVADO
5,2001586,FUNDHACRE,409010189,LITOTRIPSIA,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200004,ACRE,APROVADO
6,2001586,FUNDHACRE,409010200,NEFRECTOMIA PARCIAL,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200005,ACRE,APROVADO
7,2001586,FUNDHACRE,409010367,RESSECCAO DO COLO VESICAL / TUMOR VESICAL A CE...,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200006,ACRE,APROVADO
8,2001586,FUNDHACRE,409020176,URETROTOMIA INTERNA,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200007,ACRE,APROVADO
9,2001586,FUNDHACRE,409010170,INSTALACAO ENDOSCOPICA DE CATETER DUPLO J,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200008,ACRE,APROVADO
10,2001586,FUNDHACRE,406020078,IMPLANTAÇÃO DE CATETER DE LONGA PERMANÊNCIA SE...,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200009,ACRE,APROVADO
13,2001586,FUNDHACRE,409070050,COLPOPERINEOPLASTIA ANTERIOR E POSTERIOR,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200012,ACRE,APROVADO


### ABA 3

In [40]:
pasta = 'APROVADO'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Execução'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo_com_extensao = os.path.basename(caminho_arquivo)  # Obtém o nome do arquivo com a extensão
        nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo_com_extensao)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo_sem_extensao  # Adiciona uma coluna 'Arquivo' com o nome do arquivo sem a extensão
        df_aba1['STATUS'] = os.path.dirname(caminho_arquivo)  # Adiciona uma coluna 'Pasta' com o nome da pasta onde está o arquivo
        dfs_dict[nome_arquivo_sem_extensao] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba3 = pd.concat(dfs_dict.values(), ignore_index=True)

In [41]:
df_planilha_aba3.rename(columns={'Distribuição e Cronograma da Execução do Recurso Financeiro':'CODIGO GESTOR','Unnamed: 1':'Gestão do Recurso','Unnamed: 2':'DESCRIÇÃO DO GESTOR','Unnamed: 3':'VALOR',
                                 'Unnamed: 4':'março','Unnamed: 5':'abril','Unnamed: 6':'maio','Unnamed: 7':'junho','Unnamed: 8':'julho','Unnamed: 9':'agosto','Unnamed: 10':'setembro','Unnamed: 11':'outubro',
                                 'Unnamed: 12':'novembro','Unnamed: 13':'dezembro','Unnamed: 14':'% TOTAL','SQ (CODIGO Interno':'SQ (CODIGO Interno'}, inplace=True)

df_planilha_aba3.drop(df_planilha_aba3.index[0:6], inplace=True) # Remove as 5 primeiras linhas do arquivo
df_planilha_aba3.dropna(subset=['CODIGO GESTOR'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3.drop(df_planilha_aba3[df_planilha_aba3['CODIGO GESTOR'] == 'CODIGO GESTOR'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3.dropna(subset=['% TOTAL'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3.dropna(subset=['DESCRIÇÃO DO GESTOR'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'

cols = ["março", "abril", "maio", "junho", "julho", "agosto", "setembro", "outubro", "novembro", "dezembro", "% TOTAL"] # Define as colunas que serão formatadas
df_planilha_aba3[cols] = df_planilha_aba3[cols].fillna(0).astype(float) # Converte os valores para float
df_planilha_aba3[cols] = df_planilha_aba3[cols].applymap(lambda x: '{:.0%}'.format(float(x))) # Formata as colunas para porcentagem

format_value = lambda x: '{:,.2f}'.format(float(x)).replace('.', '-').replace(',', '.').replace('-', ',')
df_planilha_aba3['VALOR'] = df_planilha_aba3['VALOR'].apply(format_value)

df_planilha_aba3.head(10) # Exibe as 5 primeiras linhas do arquivo


,CODIGO GESTOR,Gestão do Recurso,DESCRIÇÃO DO GESTOR,VALOR,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro,% TOTAL,SQ (CODIGO Interno,UF,STATUS
6,120000,ESTADUAL,Acre,"2.550.776,40",0%,10%,15%,15%,10%,10%,10%,10%,10%,10%,100%,30001,ACRE,APROVADO
1008,270000,ESTADUAL,Alagoas,"2.835.357,05",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30001,ALAGOAS,APROVADO
1009,270430,MUNICIPAL,MACEIO,"1.453.549,63",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30002,ALAGOAS,APROVADO
1010,270860,MUNICIPAL,SAO MIGUEL DOS CAMPOS,"245.095,70",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30003,ALAGOAS,APROVADO
1011,270630,MUNICIPAL,PALMEIRA DOS INDIOS,"939.637,58",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30004,ALAGOAS,APROVADO
1012,270230,MUNICIPAL,CORURIPE,"270.060,00",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30005,ALAGOAS,APROVADO
1013,270690,MUNICIPAL,PILAR,"1.286.437,40",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30006,ALAGOAS,APROVADO
1014,270670,MUNICIPAL,PENEDO,"1.025.031,00",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30007,ALAGOAS,APROVADO
1015,270800,MUNICIPAL,SANTANA DO IPANEMA,"1.048.387,10",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30008,ALAGOAS,APROVADO
1016,270915,MUNICIPAL,TEOTONIO VILELA,"362.190,60",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30009,ALAGOAS,APROVADO


## SALVAR ARQUIVO

### EXCEL

In [42]:
writer = pd.ExcelWriter(f'PLANILHA/BASE_APROVADA.xlsx', engine='xlsxwriter') # Cria um arquivo excel

df_planilha_aba1.to_excel(writer, sheet_name='Ident. Fila na UF', index=False)
df_planilha_aba2.to_excel(writer, sheet_name='Ident. CNES e Proced.', index=False)
df_planilha_aba3.to_excel(writer, sheet_name='Execução', index=False)

writer.save()
writer.close()

### CSV

In [43]:
df_planilha_aba1.to_csv('PLANILHA/BASE_APROVADA_1.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba2.to_csv('PLANILHA/BASE_APROVADA_2.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba3.to_csv('PLANILHA/BASE_APROVADA_3.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv

# BASE NÃO APROVADAS

In [44]:
pasta = 'NAO_APROVADO'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Ident. Fila na UF'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo_com_extensao = os.path.basename(caminho_arquivo)  # Obtém o nome do arquivo com a extensão
        nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo_com_extensao)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo_sem_extensao  # Adiciona uma coluna 'Arquivo' com o nome do arquivo sem a extensão
        df_aba1['STATUS'] = os.path.dirname(caminho_arquivo)  # Adiciona uma coluna 'Pasta' com o nome da pasta onde está o arquivo
        dfs_dict[nome_arquivo_sem_extensao] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba1N = pd.concat(dfs_dict.values(), ignore_index=True)

In [45]:
df_planilha_aba1N.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA':'CÓDIGO DO PROCEDIMENTO NO SIGTAP','Unnamed: 1':'PROCEDIMENTO CIRÚRGICO',
                                 'Unnamed: 2':'QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22', 'Unnamed: 3':'Redução do Tamanho da Fila (%)', 'Unnamed: 4':'Prazo para reduzir o % proposto (em meses)', 
                                 'Unnamed: 5':'Qtde de cirurgias a serem feitas no prazo pactuado', 'Unnamed: 6':'SQ (CODIGO Interno', 'UF':'UF'} , inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'

df_planilha_aba1N.drop([0,1], inplace=True) # Exclui as linhas 0 e 1
df_planilha_aba1N.drop('Unnamed: 7', axis=1, inplace=True) # Removendo coluna 'Unnamed: 7'
df_planilha_aba1N.dropna(subset=['PROCEDIMENTO CIRÚRGICO'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1N.drop(df_planilha_aba1N[df_planilha_aba1N['PROCEDIMENTO CIRÚRGICO'] == 'TOTAL'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1N.drop(df_planilha_aba1N[df_planilha_aba1N['PROCEDIMENTO CIRÚRGICO'] == 'PROCEDIMENTO CIRÚRGICO'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba1N['Qtde de cirurgias a serem feitas no prazo pactuado'] = df_planilha_aba1N['Qtde de cirurgias a serem feitas no prazo pactuado'].astype(int) # Converte a coluna 'Qtde de cirurgias a serem feitas no prazo pactuado' para inteiro

In [46]:
df_planilha_aba1N.head(10)

,CÓDIGO DO PROCEDIMENTO NO SIGTAP,PROCEDIMENTO CIRÚRGICO,QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22,Redução do Tamanho da Fila (%),Prazo para reduzir o % proposto (em meses),Qtde de cirurgias a serem feitas no prazo pactuado,SQ (CODIGO Interno,UF,STATUS
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",166,1,9,166,100001,BAHIA,NAO_APROVADO
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,50,1,9,50,100002,BAHIA,NAO_APROVADO
4,401020088,EXERESE DE CISTO SACRO-COCCIGEO,594,0.7,9,415,100003,BAHIA,NAO_APROVADO
5,402010035,TIREOIDECTOMIA PARCIAL,22,0,9,0,100004,BAHIA,NAO_APROVADO
6,402010043,TIREOIDECTOMIA TOTAL,279,0.9,9,251,100005,BAHIA,NAO_APROVADO
7,402010051,TIREOIDECTOMIA TOTAL C/ ESVAZIAMENTO GANGLIONAR,2,1,9,2,100006,BAHIA,NAO_APROVADO
8,403020123,TRATAMENTO CIRURGICO DE SINDROME COMPRESSIVA E...,30,1,9,30,100007,BAHIA,NAO_APROVADO
9,404010016,ADENOIDECTOMIA,239,0.85,9,203,100008,BAHIA,NAO_APROVADO
10,404010024,AMIGDALECTOMIA,335,0.73,9,244,100009,BAHIA,NAO_APROVADO
11,404010032,AMIGDALECTOMIA C/ ADENOIDECTOMIA,1168,0.65,9,759,100010,BAHIA,NAO_APROVADO


## ABA 2

In [47]:
pasta = 'NAO_APROVADO'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Ident. CNES e Proced.'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo_com_extensao = os.path.basename(caminho_arquivo)  # Obtém o nome do arquivo com a extensão
        nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo_com_extensao)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo_sem_extensao  # Adiciona uma coluna 'Arquivo' com o nome do arquivo sem a extensão
        df_aba1['STATUS'] = os.path.dirname(caminho_arquivo)  # Adiciona uma coluna 'Pasta' com o nome da pasta onde está o arquivo
        dfs_dict[nome_arquivo_sem_extensao] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba2N = pd.concat(dfs_dict.values(), ignore_index=True)

In [48]:
# Tratamento dos dados da aba 'Ident. CNES e Proced.'
df_planilha_aba2N.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES':'CNES','Unnamed: 1':'ESTABELECIMENTO','Unnamed: 2':'CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP', 
                                 'Unnamed: 3':'PROCEDIMENTO CIRÚRGICO (PRINCIPAL)', 'Unnamed: 4':'COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENTO PRINCIPAL (Percentual)', 'Unnamed: 5':'GESTÃO DO SERVIÇO',
                                 'Unnamed: 6':'CODIGO DA NATUREZA JURÍDICA','Unnamed: 7':'NATUREZA JURÍDICA', 'Unnamed: 8':'POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?',
                                 'Unnamed: 9':'Identificação do Nome do Estabelecimento que não apareceu nesta planilha (coluna B)','Unnamed: 10':'SQ (CODIGO Interno'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'

df_planilha_aba2N.drop([0,1], inplace=True) # Exclui as linhas 0 e 1
df_planilha_aba2N.dropna(subset=['ESTABELECIMENTO'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba2N.drop(df_planilha_aba2N[df_planilha_aba2N['ESTABELECIMENTO'] == 'ESTABELECIMENTO'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'


In [49]:
df_planilha_aba2N.head(10)

,CNES,ESTABELECIMENTO,CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP,PROCEDIMENTO CIRÚRGICO (PRINCIPAL),COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENTO PRINCIPAL (Percentual),GESTÃO DO SERVIÇO,CODIGO DA NATUREZA JURÍDICA,NATUREZA JURÍDICA,POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?,Identificação do Nome do Estabelecimento que não apareceu nesta planilha (coluna B),SQ (CODIGO Interno,UF,STATUS
2,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200001,BAHIA,NAO_APROVADO
3,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200002,BAHIA,NAO_APROVADO
4,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,407040129,HERNIOPLASTIA UMBILICAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200003,BAHIA,NAO_APROVADO
5,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,409060135,HISTERECTOMIA TOTAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200004,BAHIA,NAO_APROVADO
6,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,409060119,HISTERECTOMIA C/ ANEXECTOMIA (UNI / BILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200005,BAHIA,NAO_APROVADO
7,2598183,HOSPITAL PORTUGUES UNIDADE REGIONAL DE C DO COITE,409060216,OOFORECTOMIA / OOFOROPLASTIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200006,BAHIA,NAO_APROVADO
8,2401231,HOSPITAL PORTUGUES UNID ANTONIO CARLOS MAGALHAES,407030026,COLECISTECTOMIA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200007,BAHIA,NAO_APROVADO
9,2401231,HOSPITAL PORTUGUES UNID ANTONIO CARLOS MAGALHAES,407030034,COLECISTECTOMIA VIDEOLAPAROSCOPICA,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200008,BAHIA,NAO_APROVADO
10,2401231,HOSPITAL PORTUGUES UNID ANTONIO CARLOS MAGALHAES,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200009,BAHIA,NAO_APROVADO
11,2401231,HOSPITAL PORTUGUES UNID ANTONIO CARLOS MAGALHAES,407040129,HERNIOPLASTIA UMBILICAL,NaN,ESTADUAL,3999,ASSOCIACAO PRIVADA,SIM,NaN,200010,BAHIA,NAO_APROVADO


## ABA 3 

In [50]:
pasta = 'NAO_APROVADO'  # Substitua pelo caminho da sua pasta

arquivos = os.listdir(pasta)  # Lista todos os nomes de arquivos na pasta

dfs_dict = {}

aba1 = 'Execução'  # Substitua pelo nome da aba que deseja ler

# Loop através de cada nome de arquivo e leitura do Excel para um DataFrame
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)  # Cria o caminho completo para o arquivo
    if os.path.isfile(caminho_arquivo) and caminho_arquivo.endswith('.xlsx'):  # Verifica se é um arquivo Excel
        nome_arquivo_com_extensao = os.path.basename(caminho_arquivo)  # Obtém o nome do arquivo com a extensão
        nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo_com_extensao)[0]  # Obtém o nome do arquivo sem a extensão
        df_aba1 = pd.read_excel(caminho_arquivo, sheet_name=aba1)  # Lê a aba 'Ident. Fila na UF' do arquivo Excel
        df_aba1['UF'] = nome_arquivo_sem_extensao  # Adiciona uma coluna 'Arquivo' com o nome do arquivo sem a extensão
        df_aba1['STATUS'] = os.path.dirname(caminho_arquivo)  # Adiciona uma coluna 'Pasta' com o nome da pasta onde está o arquivo
        dfs_dict[nome_arquivo_sem_extensao] = df_aba1  # Adiciona o DataFrame em um dicionário usando o nome do arquivo como chave

# Concatena todos os DataFrames em um único DataFrame
df_planilha_aba3N = pd.concat(dfs_dict.values(), ignore_index=True)

In [51]:
df_planilha_aba3N.rename(columns={'Distribuição e Cronograma da Execução do Recurso Financeiro':'CODIGO GESTOR','Unnamed: 1':'Gestão do Recurso','Unnamed: 2':'DESCRIÇÃO DO GESTOR','Unnamed: 3':'VALOR',
                                 'Unnamed: 4':'março','Unnamed: 5':'abril','Unnamed: 6':'maio','Unnamed: 7':'junho','Unnamed: 8':'julho','Unnamed: 9':'agosto','Unnamed: 10':'setembro','Unnamed: 11':'outubro',
                                 'Unnamed: 12':'novembro','Unnamed: 13':'dezembro','Unnamed: 14':'% TOTAL','SQ (CODIGO Interno':'SQ (CODIGO Interno'}, inplace=True)

df_planilha_aba3N.drop(df_planilha_aba3N.index[0:6], inplace=True) # Remove as 5 primeiras linhas do arquivo
df_planilha_aba3N.dropna(subset=['CODIGO GESTOR'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3N.drop(df_planilha_aba3N[df_planilha_aba3N['CODIGO GESTOR'] == 'CODIGO GESTOR'].index, inplace=True) # Exclui as linhas que possuem valor 'TOTAL' na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3N.dropna(subset=['% TOTAL'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'
df_planilha_aba3N.dropna(subset=['DESCRIÇÃO DO GESTOR'], inplace=True) # Exclui as linhas que não possuem valor na coluna 'PROCEDIMENTO CIRÚRGICO'

cols = ["março", "abril", "maio", "junho", "julho", "agosto", "setembro", "outubro", "novembro", "dezembro", "% TOTAL"] # Define as colunas que serão formatadas
df_planilha_aba3N[cols] = df_planilha_aba3N[cols].fillna(0).astype(float) # Converte os valores para float
df_planilha_aba3N[cols] = df_planilha_aba3N[cols].applymap(lambda x: '{:.0%}'.format(float(x))) # Formata as colunas para porcentagem

format_value = lambda x: '{:,.2f}'.format(float(x)).replace('.', '-').replace(',', '.').replace('-', ',')
df_planilha_aba3N['VALOR'] = df_planilha_aba3N['VALOR'].apply(format_value)


df_planilha_aba3N.head(10) # Exibe as 5 primeiras linhas do arquivo


,CODIGO GESTOR,Gestão do Recurso,DESCRIÇÃO DO GESTOR,VALOR,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro,% TOTAL,SQ (CODIGO Interno,UF,STATUS
6,290000,ESTADUAL,Bahia,"21.182.803,42",0%,10%,10%,10%,15%,10%,10%,15%,10%,10%,100%,30001,BAHIA,NAO_APROVADO
7,290570,MUNICIPAL,CAMACARI,"759.658,17",10%,10%,10%,10%,10%,10%,10%,10%,10%,10%,100%,30002,BAHIA,NAO_APROVADO
8,290600,MUNICIPAL,CAMPO FORMOSO,"176.284,29",0%,0%,20%,5%,15%,20%,20%,20%,0%,0%,100%,30003,BAHIA,NAO_APROVADO
9,291005,MUNICIPAL,DIAS D'AVILA,"205.645,35",0%,0%,15%,15%,15%,15%,20%,10%,10%,0%,100%,30004,BAHIA,NAO_APROVADO
10,291480,MUNICIPAL,ITABUNA,"526.054,58",10%,10%,10%,10%,11%,10%,10%,10%,10%,9%,100%,30005,BAHIA,NAO_APROVADO
11,291800,MUNICIPAL,JEQUIE,"383.939,29",0%,0%,20%,10%,20%,15%,10%,15%,10%,0%,100%,30006,BAHIA,NAO_APROVADO
12,291980,MUNICIPAL,MACAUBAS,"124.021,06",0%,50%,50%,0%,0%,0%,0%,0%,0%,0%,100%,30007,BAHIA,NAO_APROVADO
13,292740,MUNICIPAL,SALVADOR,"7.125.465,76",0%,15%,15%,10%,10%,10%,10%,10%,10%,10%,100%,30008,BAHIA,NAO_APROVADO
14,292880,MUNICIPAL,SANTO ESTEVAO,"131.845,93",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30009,BAHIA,NAO_APROVADO
15,292900,MUNICIPAL,SAO FELIX,"36.321,14",0%,30%,30%,40%,0%,0%,0%,0%,0%,0%,100%,30010,BAHIA,NAO_APROVADO


## SALVAR ARQUIVO NÃO APROVADO

### EXCEL

In [52]:
writer = pd.ExcelWriter(f'PLANILHA/BASE_NAO_APROVADA.xlsx', engine='xlsxwriter') # Cria um arquivo excel

df_planilha_aba1N.to_excel(writer, sheet_name='Ident. Fila na UF', index=False)
df_planilha_aba2N.to_excel(writer, sheet_name='Ident. CNES e Proced.', index=False)
df_planilha_aba3N.to_excel(writer, sheet_name='Execução', index=False)

writer.save()
writer.close()

### CSV

In [53]:
df_planilha_aba1N.to_csv('PLANILHA/BASE_NAO_APROVADA_1.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba2N.to_csv('PLANILHA/BASE_NAO_APROVADA_2.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba3N.to_csv('PLANILHA/BASE_NAO_APROVADA_3.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv

In [54]:
print('Arquivos gerados com sucesso!')

Arquivos gerados com sucesso!


# CONSOLIDAÇÃO DE ARQUIVOS

In [55]:
df_planilha_aba1G = pd.concat([df_planilha_aba1, df_planilha_aba1N], ignore_index=True)
df_planilha_aba2G = pd.concat([df_planilha_aba2, df_planilha_aba2N], ignore_index=True)
df_planilha_aba3G = pd.concat([df_planilha_aba3, df_planilha_aba3N], ignore_index=True)

In [56]:
df_planilha_aba1G.head(10) # Exibe as 5 primeiras linhas do arquivo

,CÓDIGO DO PROCEDIMENTO NO SIGTAP,PROCEDIMENTO CIRÚRGICO,QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/12/22,Redução do Tamanho da Fila (%),Prazo para reduzir o % proposto (em meses),Qtde de cirurgias a serem feitas no prazo pactuado,SQ (CODIGO Interno,UF,STATUS
0,401020029,ENXERTO DERMO-EPIDERMICO,9,0.0,NaN,0,100001,ACRE,APROVADO
1,401020037,ENXERTO LIVRE DE PELE TOTAL,14,0.0,NaN,0,100002,ACRE,APROVADO
2,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",20,0.0,NaN,0,100003,ACRE,APROVADO
3,401020053,EXCISAO E SUTURA DE LESAO NA PELE C/ PLASTICA ...,17,0.0,NaN,0,100004,ACRE,APROVADO
4,401020061,EXERESE DE CISTO BRANQUIAL,28,0.0,NaN,0,100005,ACRE,APROVADO
5,401020070,EXERESE DE CISTO DERMOIDE,138,0.0,NaN,0,100006,ACRE,APROVADO
6,401020088,EXERESE DE CISTO SACRO-COCCIGEO,1,0.0,NaN,0,100007,ACRE,APROVADO
7,401020096,EXERESE DE CISTO TIREOGLOSSO,18,1.0,9,18,100008,ACRE,APROVADO
8,401020100,EXTIRPACAO E SUPRESSAO DE LESAO DE PELE E DE T...,10,0.0,NaN,0,100009,ACRE,APROVADO
9,401020150,TRATAMENTO CIRURGICO DO SINUS PRE-AURICULAR,8,0.0,NaN,0,100010,ACRE,APROVADO


In [57]:
df_planilha_aba2G.head(10) # Exibe as 5 primeiras linhas do arquivo

,CNES,ESTABELECIMENTO,CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP,PROCEDIMENTO CIRÚRGICO (PRINCIPAL),COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENTO PRINCIPAL (Percentual),GESTÃO DO SERVIÇO,CODIGO DA NATUREZA JURÍDICA,NATUREZA JURÍDICA,POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?,Identificação do Nome do Estabelecimento que não apareceu nesta planilha (coluna B),SQ (CODIGO Interno,UF,STATUS
0,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060135,HISTERECTOMIA TOTAL,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200001,ACRE,APROVADO
1,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060186,LAQUEADURA TUBARIA,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200002,ACRE,APROVADO
2,2000296,HOSPITAL DA MULHER E DA CRIANCA DO JURUA,409060046,CURETAGEM SEMIOTICA C/ OU S/ DILATACAO DO COLO...,NaN,ESTADUAL,1023,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,NaN,NaN,200003,ACRE,APROVADO
3,2001586,FUNDHACRE,409010189,LITOTRIPSIA,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200004,ACRE,APROVADO
4,2001586,FUNDHACRE,409010200,NEFRECTOMIA PARCIAL,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200005,ACRE,APROVADO
5,2001586,FUNDHACRE,409010367,RESSECCAO DO COLO VESICAL / TUMOR VESICAL A CE...,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200006,ACRE,APROVADO
6,2001586,FUNDHACRE,409020176,URETROTOMIA INTERNA,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200007,ACRE,APROVADO
7,2001586,FUNDHACRE,409010170,INSTALACAO ENDOSCOPICA DE CATETER DUPLO J,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200008,ACRE,APROVADO
8,2001586,FUNDHACRE,406020078,IMPLANTAÇÃO DE CATETER DE LONGA PERMANÊNCIA SE...,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200009,ACRE,APROVADO
9,2001586,FUNDHACRE,409070050,COLPOPERINEOPLASTIA ANTERIOR E POSTERIOR,NaN,ESTADUAL,1147,FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL,NaN,NaN,200012,ACRE,APROVADO


In [58]:
df_planilha_aba3G.head(10)

,CODIGO GESTOR,Gestão do Recurso,DESCRIÇÃO DO GESTOR,VALOR,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro,% TOTAL,SQ (CODIGO Interno,UF,STATUS
0,120000,ESTADUAL,Acre,"2.550.776,40",0%,10%,15%,15%,10%,10%,10%,10%,10%,10%,100%,30001,ACRE,APROVADO
1,270000,ESTADUAL,Alagoas,"2.835.357,05",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30001,ALAGOAS,APROVADO
2,270430,MUNICIPAL,MACEIO,"1.453.549,63",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30002,ALAGOAS,APROVADO
3,270860,MUNICIPAL,SAO MIGUEL DOS CAMPOS,"245.095,70",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30003,ALAGOAS,APROVADO
4,270630,MUNICIPAL,PALMEIRA DOS INDIOS,"939.637,58",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30004,ALAGOAS,APROVADO
5,270230,MUNICIPAL,CORURIPE,"270.060,00",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30005,ALAGOAS,APROVADO
6,270690,MUNICIPAL,PILAR,"1.286.437,40",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30006,ALAGOAS,APROVADO
7,270670,MUNICIPAL,PENEDO,"1.025.031,00",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30007,ALAGOAS,APROVADO
8,270800,MUNICIPAL,SANTANA DO IPANEMA,"1.048.387,10",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30008,ALAGOAS,APROVADO
9,270915,MUNICIPAL,TEOTONIO VILELA,"362.190,60",0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,30009,ALAGOAS,APROVADO


# SALVAR ARQUIVO GERAL 

### EXCEL

In [59]:
writer = pd.ExcelWriter(f'PLANILHA/BASE_GERAL.xlsx', engine='xlsxwriter') # Cria um arquivo excel

df_planilha_aba1G.to_excel(writer, sheet_name='Ident. Fila na UF', index=False)
df_planilha_aba2G.to_excel(writer, sheet_name='Ident. CNES e Proced.', index=False)
df_planilha_aba3G.to_excel(writer, sheet_name='Execução', index=False)


writer.save()
writer.close()

### CSV

In [60]:
df_planilha_aba1G.to_csv('PLANILHA/BASE_GERAL_1.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba2G.to_csv('PLANILHA/BASE_GERAL_2.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv
df_planilha_aba3G.to_csv('PLANILHA/BASE_GERAL_3.csv', sep=';', encoding='latin-1', index=False) # Cria um arquivo csv

In [61]:
tempo_final = time.time()
tempo_total = int(tempo_final - tempo_inicial)

minutos = tempo_total // 60
segundos = tempo_total % 60

data_hora_atual = datetime.now() # Pega a data e hora atual
print(f"Tempo total de execução: {minutos} minutos e {segundos} segundos" ) 


Tempo total de execução: 1 minutos e 39 segundos
